In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

Referensi:
- https://arxiv.org/pdf/1608.06993.pdf
- https://towardsdatascience.com/creating-densenet-121-with-tensorflow-edbc08a956d8

In [2]:
def bn_relu_conv(x, filters, kernel_size=1, strides=1, padding='same'):
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    return x

![Tabel DenseNet](https://amaarora.github.io/images/densenet_archs.png)

In [3]:
# DenseNet Layer Template


def DenseNet(num_classes: int, nb_layers:list[int], growth_rate:int=32, ) -> tf.Tensor:
    input = layers.Input((224, 224, 3))
    ConvolutionLayer = layers.Conv2D(64, (3, 3), padding='same')
    PoolingLayer = layers.MaxPooling2D((3,3), strides=2)
    def TransitionLayer(x: tf.Tensor) -> tf.Tensor:
        x = bn_relu_conv(x, filters=64, kernel_size=(1,1))
        x = layers.AveragePooling2D((2,2), strides=2)(x)
        return x

    def DenseBlock(x: tf.Tensor, nb_layers: int) -> tf.Tensor:
        for i in range(nb_layers):
            y = bn_relu_conv(x, filters=growth_rate*4, kernel_size=1)
            y = bn_relu_conv(y, filters=growth_rate, kernel_size=3)
            x = layers.concatenate([x, y])
        return x

    def ClassificationLayer(x: tf.Tensor, num_classes) -> tf.Tensor:
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(num_classes, activation=('softmax'))(x)
        return x
    
    x = ConvolutionLayer(input)
    x = PoolingLayer(x)
    for i in range(len(nb_layers)):
        d = DenseBlock(x, nb_layers[i])
        x = TransitionLayer(d)
    output = ClassificationLayer(d, num_classes)

    model = models.Model(input, output)
    return model

In [4]:
DenseNet121 = DenseNet(2, [6,12,24,16])
# DenseNet169 = DenseNet(2, [6,12,32,32])
# DenseNet201 = DenseNet(2, [6,12,48,32])
# DenseNet264 = DenseNet(2, [6,12,64,64])
DenseNet121.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dataset

In [5]:
import numpy.random as rnd
seed = rnd.randint(1, 10000)
print("Seed is",seed)
train_ds = tf.keras.preprocessing.image_dataset_from_directory('D:\AI-Project\Pancarona PPSMB UGM - Gender', validation_split=0.2, subset='training', seed=seed, image_size=(224,224), batch_size=16, label_mode='int')
val_ds = tf.keras.preprocessing.image_dataset_from_directory('D:\AI-Project\Pancarona PPSMB UGM - Gender', validation_split=0.2, subset='validation', seed=seed, image_size=(224,224), batch_size=16, label_mode='int')

Found 9506 files belonging to 2 classes.
Using 7605 files for training.
Found 9506 files belonging to 2 classes.
Using 1901 files for validation.


# Training

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os



timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'densenet121-'+timestr

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1
)
"""
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/'+name,
    histogram_freq=1
)
"""
DenseNet121.fit(
    train_ds,
    epochs=10,
    verbose=1,
    validation_data= val_ds,
    batch_size=16,
    callbacks=[cp_callback]
    #callbacks=[cp_callback, tensorboard_callback]
)

Epoch 1/10
 18/476 [>.............................] - ETA: 3:07 - loss: 1.9582 - accuracy: 0.6319

In [ ]:
model.save('gender-classifier-ugm')

In [ ]:
tf.math.confusion_matrix(
    labels,
    predictions,
    num_classes=None,
    weights=None,
    dtype=tf.dtypes.int32,
    name=None
)
